In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1) 행정동별 문화시설 수 분석

In [47]:
df_culture = pd.read_csv('서울시 문화공간 정보.csv', encoding='cp949');

# 정보 확인
# print(df_population.info())
# print(df_culture.info)

# 필요한 데이터만 추출

df_culture = df_culture[['주제분류', '문화시설명', '주소', '위도', '경도', '홈페이지', '대표이미지', '시설소개']]
# print(df_culture_selected.info())

# 주제 카테고리 확인
culture_category = df_culture['주제분류'].unique()
# print(culture_category)

# 결측치 확인 및 제거
df_culture = df_culture.dropna(subset=['주소'])
# print(df_culture.isnull().sum())

# 중복값 확인 및 제거
df_culture = df_culture.drop_duplicates(subset=['주소'])
df_culture_duplicated = df_culture[df_culture.duplicated(subset=['주소'])]
print('중복되는 주소: ', df_culture_duplicated) # 중복 없는거 확인

df_culture.to_excel('서울시 문화공간 정보_가공.xlsx') # 내보낸 후 도로명-> 지번으로 2차 가공

중복되는 주소:  Empty DataFrame
Columns: [주제분류, 문화시설명, 주소, 위도, 경도, 홈페이지, 대표이미지, 시설소개]
Index: []


In [107]:
df_culture_new = pd.read_excel('서울시 문화공간 정보_가공.xlsx')
df_population = pd.read_csv('지역별(법정동) 성별 연령별 주민등록 인구수_20231231.csv', encoding='cp949')

# 이상값 및 결측치 처리
df_culture_new = df_culture_new[df_culture_new['주소'].str.startswith('서울')]
df_culture_new = df_culture_new.dropna(subset=['주소'])

df_culture_new['자치구'] = df_culture_new['주소'].apply(lambda x: x.split()[1])
df_culture_new['행정동'] = df_culture_new['주소'].apply(lambda x: x.split()[2])
print(df_culture_new)

df_culture_new.to_csv('가공_서울시 문화공간 정보(+자치구&행정동).csv')

# 행정동별 문화공간 수만 있는 데이터 프레임 가공
df_culture_count = df_culture_new['행정동'].value_counts()
df_culture_count = pd.DataFrame({'총 문화시설 수': df_culture_count})

df_population = df_population[df_population['시도명'] == '서울특별시']
df_population.rename(columns={'읍면동명': '행정동', '계': '총 인구수'}, inplace=True)


# 두 데이터프레임을 조인
merged_df = pd.merge(df_culture_count, df_population, on='행정동', how='left')

df_culture_count = merged_df[['행정동', '총 문화시설 수', '총 인구수']]

df_culture_count['1인당 문화시설 비율'] = df_culture_count['총 문화시설 수'] / df_culture_count['총 인구수']
df_culture_count['1인당 문화시설 백분율'] = df_culture_count['1인당 문화시설 비율'] * 100

print(df_culture_count)

df_culture_count.to_csv('행정동별 문화시설 수.csv')

     Unnamed: 0    주제분류           문화시설명                   주소         위도  \
0             0      기타           경희대학교      서울 동대문구 회기동 1-5  37.593949   
1             1      기타  DDP(동대문디자인플라자)      서울 중구 을지로7가 143  37.566813   
2             2     공연장            우란2경  서울 성동구 성수동2가 314-12  37.543790   
3             3     도서관      구립은평뉴타운도서관        서울 은평구 진관동 93  37.637230   
4             4  문화예술회관        노원문화예술회관     서울 노원구 중계동 364-3  37.650237   
..          ...     ...             ...                  ...        ...   
913         991      기타            서초구청    서울 서초구 서초동 1376-3  37.483625   
914         992     도서관         중랑상봉도서관       서울 중랑구 상봉동 126  37.592892   
915         993      기타           중랑캠핑숲        서울 중랑구 망우동 56  37.606861   
916         996      기타          정동제일교회        서울 중구 정동 34-3  37.565183   
917         997      기타    롯데시네마 가산디지털점      서울 금천구 가산동 60-8  37.477619   

             경도                                               홈페이지  \
0    127.054891            ht

<ipython-input-107-29c35e7c1568>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_culture_count['1인당 문화시설 비율'] = df_culture_count['총 문화시설 수'] / df_culture_count['총 인구수']
<ipython-input-107-29c35e7c1568>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_culture_count['1인당 문화시설 백분율'] = df_culture_count['1인당 문화시설 비율'] * 100


# 2) 행정동별 병의원 및 약국 수 분석


In [14]:
df_pharmacy = pd.read_csv('서울시 약국 인허가 정보.csv', encoding='cp949');

# 정보 확인
# print(df_pharmacy.info())

df_pharmacy = df_pharmacy[['상세영업상태명', '지번주소', '도로명주소', '사업장명', '최종수정일자', '좌표정보(X)', '좌표정보(Y)']]
df_pharmacy = df_pharmacy[(df_pharmacy['상세영업상태명'].isin(['영업중', '휴업', '사용중'])) & (df_pharmacy['최종수정일자'] < '2024-01-01')]
# print(df_pharmacy)
# print(df_pharmacy['상세영업상태명'].unique())

# 결측치 확인
print(df_pharmacy.isnull().sum())
df_pharmacy = df_pharmacy.dropna(subset=['도로명주소'])
print(df_pharmacy.isnull().sum())

# 중복값 확인 및 제거
df_pharmacy = df_pharmacy.drop_duplicates(subset=['지번주소', '도로명주소'])
df_pharmacy_duplicated = df_pharmacy[df_pharmacy.duplicated(subset=['지번주소', '도로명주소'])]
print('중복되는 주소: ', df_pharmacy_duplicated) # 중복 없는거 확인

df_pharmacy.to_excel('서울시 약국 정보_가공.xlsx') # 내보낸 후 도로명-> 지번으로 2차 가공

상세영업상태명      0
지번주소       273
도로명주소        2
사업장명         0
최종수정일자       0
좌표정보(X)     76
좌표정보(Y)     76
dtype: int64
상세영업상태명      0
지번주소       273
도로명주소        0
사업장명         0
최종수정일자       0
좌표정보(X)     74
좌표정보(Y)     74
dtype: int64
중복되는 주소:  Empty DataFrame
Columns: [상세영업상태명, 지번주소, 도로명주소, 사업장명, 최종수정일자, 좌표정보(X), 좌표정보(Y)]
Index: []


In [15]:
df_hospital = pd.read_csv('서울시 의원 인허가 정보.csv', encoding='cp949');

df_hospital = df_hospital[['상세영업상태명', '지번주소', '도로명주소', '사업장명', '최종수정일자', '좌표정보(X)', '좌표정보(Y)']]
df_hospital = df_hospital[(df_hospital['상세영업상태명'].isin(['영업중', '휴업', '사용중'])) & (df_hospital['최종수정일자'] < '2024-01-01')]

# 결측치 확인
df_hospital = df_pharmacy.dropna(subset=['도로명주소'])
print(df_hospital.isnull().sum())

# 중복값 확인 및 제거
df_hospital = df_hospital.drop_duplicates(subset=['지번주소', '도로명주소'])
df_hospital_duplicated = df_hospital[df_hospital.duplicated(subset=['지번주소', '도로명주소'])]
print('중복되는 주소: ', df_hospital_duplicated) # 중복 없는거 확인

df_hospital.to_excel('서울시 병원 정보_가공.xlsx') # 내보낸 후 도로명-> 지번으로 2차 가공

<ipython-input-15-a886dbd29b61>:1: DtypeWarning: Columns (3,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_hospital = pd.read_csv('서울시 의원 인허가 정보.csv', encoding='cp949');


상세영업상태명      0
지번주소       273
도로명주소        0
사업장명         0
최종수정일자       0
좌표정보(X)     74
좌표정보(Y)     74
dtype: int64
중복되는 주소:  Empty DataFrame
Columns: [상세영업상태명, 지번주소, 도로명주소, 사업장명, 최종수정일자, 좌표정보(X), 좌표정보(Y)]
Index: []


In [105]:
df_population = pd.read_csv('지역별(법정동) 성별 연령별 주민등록 인구수_20231231.csv', encoding='cp949')
df_pharmacy_new = pd.read_excel('서울시 약국 정보_가공.xlsx')
df_hospital_new = pd.read_excel('서울시 병원 정보_가공.xlsx')


df_pharmacy_new = df_pharmacy_new.dropna(subset=['지번주소'])
df_hospital_new = df_hospital_new.dropna(subset=['지번주소'])

df_pharmacy_new['자치구'] = df_pharmacy_new['지번주소'].apply(lambda x: x.split(" ")[1])
df_pharmacy_new['행정동'] = df_pharmacy_new['지번주소'].apply(lambda x: x.split(" ")[2])

df_hospital_new['자치구'] = df_hospital_new['지번주소'].apply(lambda x: x.split(" ")[1])
df_hospital_new['행정동'] = df_hospital_new['지번주소'].apply(lambda x: x.split(" ")[2])

df_pharmacy_new.to_csv('서울시 약국 정보(+자치구,행정동 분리).csv')
df_hospital_new.to_csv('서울시 의원 정보(+자치구,행정동 분리).csv')

# 행정동별 문화공간 수만 있는 데이터 프레임 가공
df_pharmacy_count = df_pharmacy_new['행정동'].value_counts()
df_hospital_count = df_hospital_new['행정동'].value_counts()
# pharmacy_hospital_per_population = df_pharmacy_hospital_count / total_population
df_pharmacy_hospital_count = pd.DataFrame({'총 병의원 및 약국 수': df_pharmacy_count + df_hospital_count})

df_population = df_population[df_population['시도명'] == '서울특별시']
df_population.rename(columns={'읍면동명': '행정동', '계': '총 인구수'}, inplace=True)

# 두 데이터프레임을 조인
merged_df = pd.merge(df_pharmacy_hospital_count, df_population, on='행정동', how='left')

df_pharmacy_hospital_count = merged_df[['행정동', '총 병의원 및 약국 수', '총 인구수']]

df_pharmacy_hospital_count['1인당 병의원 및 약국 비율'] = df_pharmacy_hospital_count['총 병의원 및 약국 수'] / df_pharmacy_hospital_count['총 인구수']
df_pharmacy_hospital_count['1인당 병의원 및 약국 백분율'] = df_pharmacy_hospital_count['1인당 병의원 및 약국 비율'] * 100

print(df_pharmacy_hospital_count)


df_pharmacy_hospital_count.to_csv('행정동별 병의원 및 약국 수.csv')

       행정동  총 병의원 및 약국 수     총 인구수  1인당 병의원 및 약국 비율  1인당 병의원 및 약국 백분율
0      상계동           212  197542.0         0.001073          0.107319
1      서초동           212  106213.0         0.001996          0.199599
2      봉천동           204  221775.0         0.000920          0.091985
3      신림동           202  242494.0         0.000833          0.083301
4      역삼동           192   70529.0         0.002722          0.272228
..     ...           ...       ...              ...               ...
349  을지로1가             2       1.0         2.000000        200.000000
350    용문동             2    6101.0         0.000328          0.032782
351     초동             2      76.0         0.026316          2.631579
352  봉래동2가             2       NaN              NaN               NaN
353  충무로5가             2     594.0         0.003367          0.336700

[354 rows x 5 columns]


<ipython-input-105-20d2cb8eb0c7>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pharmacy_new['자치구'] = df_pharmacy_new['지번주소'].apply(lambda x: x.split(" ")[1])
<ipython-input-105-20d2cb8eb0c7>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pharmacy_new['행정동'] = df_pharmacy_new['지번주소'].apply(lambda x: x.split(" ")[2])
<ipython-input-105-20d2cb8eb0c7>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.p

# 3) 행정동별 청소년, 청년, 고령 비율 분석

In [103]:
#인구 관련 데이터 분석
df_people = pd.read_csv('지역별(법정동) 성별 연령별 주민등록 인구수_20231231.csv', encoding='cp949')

# NaN 값을 0으로 채우기
df_people.fillna(0, inplace=True)

# '-' 값을 0으로 대체
df_people.replace('-', 0, inplace=True)

# 데이터프레임 정보 확인
df_people = df_people[df_people['시도명'] == '서울특별시']
df_people_new = df_people[['읍면동명', '계']]
df_people_new.rename(columns={'읍면동명': '행정동', '계': '총 인구수'}, inplace=True)

print(df_people_new)

df_people_new['청소년층(0~19세)'] = df_people[['0세남자', '0세여자', '1세남자', '1세여자', '2세남자', '2세여자',
                               '3세남자', '3세여자', '4세남자', '4세여자', '5세남자', '5세여자',
                               '6세남자', '6세여자', '7세남자', '7세여자', '8세남자', '8세여자',
                               '9세남자', '9세여자', '10세남자', '10세여자', '11세남자', '11세여자',
                               '12세남자', '12세여자', '13세남자', '13세여자', '14세남자', '14세여자',
                               '15세남자', '15세여자', '16세남자', '16세여자', '17세남자', '17세여자',
                               '18세남자', '18세여자', '19세남자', '19세여자']].sum(axis=1)

df_people_new['청년층(20~34세)'] = df_people[['20세남자', '20세여자', '21세남자', '21세여자', '22세남자', '22세여자',
                               '23세남자', '23세여자', '24세남자', '24세여자', '25세남자', '25세여자',
                               '26세남자', '26세여자', '27세남자', '27세여자', '28세남자', '28세여자',
                               '29세남자', '29세여자', '30세남자', '30세여자', '31세남자', '31세여자',
                               '32세남자', '32세여자', '33세남자', '33세여자', '34세남자', '34세여자']].sum(axis=1)

df_people_new['노년층(65세 이상)'] = df_people[['65세남자', '65세여자', '66세남자', '66세여자', '67세남자', '67세여자',
                                 '68세남자', '68세여자', '69세남자', '69세여자', '70세남자', '70세여자',
                                 '71세남자', '71세여자', '72세남자', '72세여자', '73세남자', '73세여자',
                                 '74세남자', '74세여자', '75세남자', '75세여자', '76세남자', '76세여자',
                                 '77세남자', '77세여자', '78세남자', '78세여자', '79세남자', '79세여자',
                                 '80세남자', '80세여자', '81세남자', '81세여자', '82세남자', '82세여자',
                                 '83세남자', '83세여자', '84세남자', '84세여자', '85세남자', '85세여자',
                                 '86세남자', '86세여자', '87세남자', '87세여자', '88세남자', '88세여자',
                                 '89세남자', '89세여자', '90세남자', '90세여자', '91세남자', '91세여자',
                                 '92세남자', '92세여자', '93세남자', '93세여자', '94세남자', '94세여자',
                                 '95세남자', '95세여자', '96세남자', '96세여자', '97세남자', '97세여자',
                                 '98세남자', '98세여자', '99세남자', '99세여자', '100세남자', '100세여자',
                                 '101세남자', '101세여자', '102세남자', '102세여자', '103세남자', '103세여자',
                                 '104세남자', '104세여자', '105세남자', '105세여자', '106세남자', '106세여자',
                                 '107세남자', '107세여자', '108세남자', '108세여자', '109세남자', '109세여자',
                                 '110세이상 남자', '110세이상 여자']].sum(axis=1)


print(df_people_new)

df_people_new['청소년층(0~19세) 백분율'] = df_people_new['청소년층(0~19세)'] / df_people_new['총 인구수'] * 100
df_people_new['청년층(20~34세) 백분율'] = df_people_new['청년층(20~34세)'] / df_people_new['총 인구수'] * 100
df_people_new['노년층(65세~) 백분율'] = df_people_new['노년층(65세 이상)'] / df_people_new['총 인구수'] * 100

print(df_people_new)

df_people_new.to_csv('총 인구수 대비 청소년층, 청년층, 노년층 백분율.csv')

     행정동  총 인구수
0    청운동   2289
1    신교동   1340
2    궁정동    211
3    효자동    570
4    창성동    404
..   ...    ...
450  둔촌동  24349
451  암사동  65775
452  성내동  65333
453  천호동  84811
454  강일동  34099

[455 rows x 2 columns]
     행정동  총 인구수  청소년층(0~19세)  청년층(20~34세)  노년층(65세 이상)
0    청운동   2289          432          414          414
1    신교동   1340          202          269          223
2    궁정동    211           41           41           16
3    효자동    570           75          113           91
4    창성동    404           49           67           92
..   ...    ...          ...          ...          ...
450  둔촌동  24349         3374         5037         4581
451  암사동  65775         9601        12545        12264
452  성내동  65333         7249        15749        12420
453  천호동  84811         7920        20785        17143
454  강일동  34099         6789         5430         6221

[455 rows x 5 columns]
     행정동  총 인구수  청소년층(0~19세)  청년층(20~34세)  노년층(65세 이상)  청소년층(0~19세) 백분율  \
0    청운동   2289          

<ipython-input-103-215bde94124f>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_people_new.rename(columns={'읍면동명': '행정동', '계': '총 인구수'}, inplace=True)
<ipython-input-103-215bde94124f>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_people_new['청소년층(0~19세)'] = df_people[['0세남자', '0세여자', '1세남자', '1세여자', '2세남자', '2세여자',
<ipython-input-103-215bde94124f>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.